# Python egm adapter

1) Python version 2.7

So, the point of an adapter program is to communicate with the egm server (which is the cap I'm putting on top of the robot controller). To do this, you're gonna have to use google protocol buffers to serialize and deseralize the messages sent between the server and this addapter. Because RobotStudio uses v2.4.1 protocol buffers, the decision was made to also use the same version for the rest of the communication. The problem is that v2.4.1 protocol buffers aren't supported in Python 3. So... if you're going to write an adapter in python, you're gonna have to use python 2. I'm sorry for that, but it is what is it.

2) Python environment setup

Because we're cobbeling together old stuff together, there's a couple of things you might have to do to make your python2 environment work with google protocol buffers. The first to mention is that installing everything in your environment might confuse python about where the standard print output is and what the standard encoding is. I'm not completely sure why, but o fix that, I've added some resetting script at the begenning of everything. The second thing is the actual instalation of the libraries you'll need. Everything below that is commented out are the instalation commands. I suggest updating pip and setuptools first, then uninstalling google in your environment. Once you do that, try to run the force install of protocol buffers v2.4.1. That should work. If you're running these scripts in a jupyter notebook like I am, you can just uncomment the commands that start with: !{sys.executable} -m... If you're using this code to make a command line script or something, just run the command in your python environment (i.e. pip uninstall google) without the !{sys.executable} -m part. 

In [1]:
# this is a test path... the plan is to use this path to control 
# the robot in position guidance
target0 = [-0.05758161, -0.056587974, 10.010103493 ]
target1 = [299.94241839, -0.056587974, 10.010103493]
target2 = [299.94241839, -200.056587974, 10.010103493]
target3 = [213.474183961, -200.056587974, 10.010103493]
target4 = [190.254237086, -126.401926961, 10.010103493]
target5 = [122.459604485, -163.389260093, 10.010103493]
target6 = [59.039406438, -163.389260093, 10.010103493]
target7 = [33.084307566, -142.780034243, 10.010103493]
target8 = [-0.05758161, -0.056587974, 10.010103493]
path = []
path.append(target0)
path.append(target1)
path.append(target2)
path.append(target3)
path.append(target4)
path.append(target5)
path.append(target6)
path.append(target7)
path.append(target8)

In [2]:
t0 = [878.795086254,-300.066190776,1412.499981377]
t1 = [886.791796692,275.537350992,1412.499981377]
path2 = []
path2.append(t0)
path2.append(t1)

In [3]:
import sys
stdout = sys.stdout
reload(sys)
sys.setdefaultencoding('utf-8')
sys.stdout = stdout

#!{sys.executable} -m pip install numpy

# you may need to uninstall google because it causes conflicts with the protobuf v2.4.1
#!{sys.executable} -m pip uninstall google

# install protobuf v2.4.1
#!{sys.executable} -m pip install protobuf==2.4.1 --force-reinstall

# you might need to upgrade your package manager stuff
#!{sys.executable} -m pip install --upgrade pip
#!{sys.executable} -m pip install --upgrade setuptools

import time
import threading
import numpy as np
#import scipy as sc
import socket
import test_pb2 as test_proto
import lth_egm_interface_pb2 as interface_proto
import line_sensor_pb2 as sensor

In [4]:
UDP_IP_ADDRESS = "127.0.0.1"
UDP_PORT_NO = 8081             #this is probably irrelivant 
UDP_SERVER_PORT_NO = 8080
SENSOR_PORT_NO = 12345

In [5]:
class DataStructure():
    """This is where I'm going to hold the data coming into the Model"""
    
    def __init__(self):
        """Initialize the data structure with a mutex"""
        self.mutex = threading.Lock()
        #sensor data stuff
        self.sensors = {}
        self.oldY = 0.0
        self.newY = 0.0
        self.deltaY = 0.0
        #robot position stuff
        self.curX = 0.0
        self.curY = 0.0
        self.curZ = 0.0
        self.nextX = 0.0
        self.nextY = 0.0
        self.nextZ = 0.0
    
    def processSensorData(self, data):
        """Process data from a sensor"""
        self.mutex.acquire()
        #print "sensor data called"
            #if 'key1' in dict:
              #print "blah"
            #else:
              #print "boo"
        #print "data.sensorID: %s"%data.sensorID
        if data.sensorID == 1:
            print "sensorID = %s"%data.sensorID
            if self.oldY == 0.0 and self.newY == 0.0:
                print "ID NOT in sensors"
                self.newY = data.sensedPoint.y
            else:
                print "ID in sensors"
                self.oldY = self.newY
                self.newY = data.sensedPoint.y
                self.deltaY = self.newY - self.oldY
            print "new: %s , old: %s , delta: %s"%(self.newY, self.oldY, self.deltaY)
        #self.sensors[data.sensorID] = data
        #for state in sensors.values():
            #print state
        self.mutex.release()
            
    def processEGMData(self, data):
        """Process data from egm adapter"""
        self.mutex.acquire()
        print "--------------EGM DATA--------------"
        #print data
        #testControl = control.EGM_Control()
        #testControl.currentPosition.cartesian_x = 1.0
        #testControl.currentPosition.cartesian_y = 2.0
        #testControl.currentPosition.cartesian_z = 3.0
        self.curX = data.currentPosition.cartesian_x
        self.curY = data.currentPosition.cartesian_y
        self.curZ = data.currentPosition.cartesian_z
        print "(x:%s, y:%s, z:%s)"%(self.curX, self.curY, self.curZ)
        self.mutex.release()
        
    def calculateNextPose(self):
        #print "not implemented"
        self.mutex.acquire()
        nextPose = make_target(self.curX, self.curY + self.deltaY, self.curZ)
        print nextPose
        self.mutex.release()
        return nextPose

In [6]:
def make_undefined_message():
    """Make an undefined EGM_Control message. These messages are used to ping the server to see if it is running."""
    ping = interface_proto.EGM_Control()
    ping.header.mtype = interface_proto.Header.MSGTYPE_UNDEFINED
    return ping
    

def make_test_request():
    """populate an EGM_Control message with data for a request of all current robot data"""
    request = interface_proto.EGM_Control()
    request.header.mtype = interface_proto.Header.MSGTYPE_REQUEST_ALL_VALUES
    return request

def make_target_command(target):
    """populate an EGM_Control message with data to provide a new desired target"""
    command = interface_proto.EGM_Control()
    command.header.mtype = interface_proto.Header.MSGTYPE_POS_COMMAND
    command.desiredPosition.cartesian_x = target[0]
    command.desiredPosition.cartesian_y = target[1]
    command.desiredPosition.cartesian_z = target[2]
    return command

def make_target(x, y, z):
    """make a target... simple right now, will include more than x, y, z in future... but I want to standardize what a target is"""
    # SHOULD REALLY USE NUMPY FOR ALL THIS STUFF ANYWAY... I'LL ADD THAT TO THE DO-DO LIST TOO 
    target = [x, y, z,]
    return target

def check_target_convergence(targ1, targ2, conv_criteria):
    """check the euclidean distance between the two targets and compare to convergence criteria"""
    # will check if difference between the x, y, z of target 1 and target 2 are within the distance given by the convergence criteria
    p1 = np.array([targ1[0], targ1[1], targ1[2]])
    p2 = np.array([targ2[0], targ2[1], targ2[2]])
    dist = np.linalg.norm(p1-p2)
    #print dist
    if dist <= conv_criteria:
        return True
    else:
        return False

def send_interface_message(server_address, interface_message):
    """send an EGM_Control message to the server and return the response"""
    client_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
    response = None
    try:
        sent = client_socket.sendto(interface_message.SerializeToString(), server_address)
        response_data, server = client_socket.recvfrom(4096)
        response = interface_proto.EGM_Control()
        if len(response_data) is not None:
            response.ParseFromString(response_data)
        else:
            print "no response"
    finally:
        client_socket.close()
        return response

In [7]:
# THIS IS THE SERVER LISTENING FOR SENSOR DATA
def sensor_listener(dataStructure):
    sensorSock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
    sensorSock.bind((UDP_IP_ADDRESS, SENSOR_PORT_NO))
    runSensor = True
    print "sensor server started"
    while runSensor:
        data, addr = sensorSock.recvfrom(1024)
        if len(data) is not None:
            sensorData = sensor.LineSensor()
            sensorData.ParseFromString(data)
            #print sensorData.sensedPart
            #--------------------monitor-------------------
            dataStructure.processSensorData(sensorData)
            #--------------------monitor-------------------
        else:
            print "no sensor data" 
    return

# THIS IS THE SERVER FOR THE DEMO
def demo_server(dataStructure):
    client_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
    server_address = (UDP_IP_ADDRESS, UDP_SERVER_PORT_NO)
    runDemo = False
    pingServer = True
    attempt = 1
    print "demo server started"
    while pingServer:
        ping = make_undefined_message()
        ping_response = send_interface_message(server_address, ping)
        print "ping attempt: %s"%attempt
        attempt = attempt + 1
        if ping_response is not None:
            if ping_response.header.mtype == interface_proto.Header.MSGTYPE_UNDEFINED:
                runDemo = True
                pingServer = False
        else:
            time.sleep(0.5)
            
    time.sleep(0.1)
    while runDemo:
        print "RUNNING DEMO"
        position_request = make_test_request()
        position_response = send_interface_message(server_address, position_request)
        dataStructure.processEGMData(position_response)
        time.sleep(0.5)
        

# THIS IS THE SERVER FOR MY PROTOCOL 
def greg_protocol_client(path, dataStructure):
    client_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
    server_address = (UDP_IP_ADDRESS, UDP_SERVER_PORT_NO)
    conv_criteria = 0.1 #I have no idea... just a number right now
    print "guide start"
    for target in path:
        print "target start"
        # make target command
        target_message = make_target_command(target)
        # make sure that it won't start sending all sorts of requests nothing is returned
        start_checking_convergence = False
        # send the target command and wait for the response
        target_response = send_interface_message(server_address, target_message)
        #print "target response:"
        #print target_response
        #--------------------monitor-------------------
        dataStructure.processEGMData(target_response)
        #--------------------monitor-------------------
        # if the response is valid, start checking if the robot has reached the target
        if target_response is not None:
            start_checking_convergence = True
        while start_checking_convergence:
            position_request = make_test_request()
            position_response = send_interface_message(server_address, position_request)
            #print "position response:"
            #print position_response
            #--------------------monitor-------------------
            dataStructure.processEGMData(position_response)
            #--------------------monitor-------------------
            if position_response is not None:
                current_position = make_target(position_response.currentPosition.cartesian_x, position_response.currentPosition.cartesian_y, position_response.currentPosition.cartesian_z)
                if check_target_convergence(target, current_position, conv_criteria):
                    start_checking_convergence = False
            else:
                start_checking_convergence = False
            time.sleep(0.1) #Sleep 100 milliseconds 
    
    # WHEN DONE PRINT DONE
    print "DONE!"
    return

In [8]:
monitor = DataStructure()

In [9]:
sensor_thread = threading.Thread(target=sensor_listener, args=(monitor,))
sensor_thread.start()
demo_thread = threading.Thread(target=demo_server, args=(monitor,))
demo_thread.start()

sensor server started
demo server started
ping attempt: 1


In [10]:
#path_thread = threading.Thread(target=greg_protocol_client, args=(path2, monitor,))
#path_thread.start()

ping attempt: 2
ping attempt: 3
ping attempt: 4
ping attempt: 5
ping attempt: 6
ping attempt: 7
ping attempt: 8
ping attempt: 9
ping attempt: 10
ping attempt: 11
ping attempt: 12
ping attempt: 13
ping attempt: 14
ping attempt: 15
ping attempt: 16
ping attempt: 17
ping attempt: 18
ping attempt: 19
ping attempt: 20
ping attempt: 21
ping attempt: 22
ping attempt: 23
ping attempt: 24
ping attempt: 25
ping attempt: 26
ping attempt: 27
ping attempt: 28
ping attempt: 29
sensorID = 1
ID NOT in sensors
new: -0.154459197998 , old: 0.0 , delta: 0.0
sensorID = 1
ID in sensors
new: -0.145577255249 , old: -0.154459197998 , delta: 0.00888194274902
sensorID = 1
ID in sensors
new: -0.136697219849 , old: -0.145577255249 , delta: 0.00888003540039
sensorID = 1
ID in sensors
new: -0.127820373535 , old: -0.136697219849 , delta: 0.00887684631348
sensorID = 1
ID in sensors
new: -0.118948020935 , old: -0.127820373535 , delta: 0.0088723526001
sensorID = 1
ID in sensors
new: -0.110081420898 , old: -0.11894802093

sensorID = 1
ID in sensors
new: -0.0945620727539 , old: -0.103566497803 , delta: 0.00900442504883
sensorID = 1
ID in sensors
new: -0.0856016235352 , old: -0.0945620727539 , delta: 0.00896044921875
sensorID = 1
ID in sensors
new: -0.0767099914551 , old: -0.0856016235352 , delta: 0.00889163208008
sensorID = 1
ID in sensors
new: -0.0677879638672 , old: -0.0767099914551 , delta: 0.00892202758789
sensorID = 1
ID in sensors
new: -0.0588368225098 , old: -0.0677879638672 , delta: 0.00895114135742
sensorID = 1
ID in sensors
new: -0.0498578491211 , old: -0.0588368225098 , delta: 0.00897897338867
sensorID = 1
ID in sensors
new: -0.0498578491211 , old: -0.0498578491211 , delta: 0.0
sensorID = 1
ID in sensors
new: 0.0 , old: -0.0498578491211 , delta: 0.0498578491211
RUNNING DEMO
--------------EGM DATA--------------
(x:922.869018555, y:1.37976336479, z:1407.03967285)
sensorID = 1
ID in sensors
new: 0.0 , old: 0.0 , delta: 0.0
sensorID = 1
ID NOT in sensors
new: -0.175884277344 , old: 0.0 , delta: 0.

RUNNING DEMO
--------------EGM DATA--------------
(x:922.869262695, y:1.37872314453, z:1407.04003906)
sensorID = 1
ID in sensors
new: 0.0 , old: 0.0 , delta: 0.0
sensorID = 1
ID NOT in sensors
new: -0.179419143677 , old: 0.0 , delta: 0.0
sensorID = 1
ID in sensors
new: -0.170756744385 , old: -0.179419143677 , delta: 0.00866239929199
sensorID = 1
ID in sensors
new: -0.162049758911 , old: -0.170756744385 , delta: 0.00870698547363
sensorID = 1
ID in sensors
new: -0.153299407959 , old: -0.162049758911 , delta: 0.00875035095215
sensorID = 1
ID in sensors
new: -0.144506958008 , old: -0.153299407959 , delta: 0.00879244995117
sensorID = 1
ID in sensors
new: -0.135673675537 , old: -0.144506958008 , delta: 0.0088332824707
sensorID = 1
ID in sensors
new: -0.126800842285 , old: -0.135673675537 , delta: 0.00887283325195
sensorID = 1
ID in sensors
new: -0.117889755249 , old: -0.126800842285 , delta: 0.00891108703613
sensorID = 1
ID in sensors
new: -0.108941680908 , old: -0.117889755249 , delta: 0.00

RUNNING DEMO
--------------EGM DATA--------------
(x:922.86920166, y:1.37955272198, z:1407.04003906)
RUNNING DEMO
--------------EGM DATA--------------
(x:922.869384766, y:1.37799453735, z:1407.03991699)
RUNNING DEMO
--------------EGM DATA--------------
(x:922.869506836, y:1.37908017635, z:1407.0402832)
RUNNING DEMO
--------------EGM DATA--------------
(x:922.870117188, y:1.37744665146, z:1407.04089355)
RUNNING DEMO
--------------EGM DATA--------------
(x:922.869750977, y:1.37890589237, z:1407.04089355)
RUNNING DEMO
--------------EGM DATA--------------
(x:922.868896484, y:1.37958538532, z:1407.03918457)
RUNNING DEMO
--------------EGM DATA--------------
(x:922.869262695, y:1.37860989571, z:1407.03967285)
RUNNING DEMO
--------------EGM DATA--------------
(x:922.86932373, y:1.37902128696, z:1407.04003906)
sensorID = 1
ID in sensors
new: 0.0 , old: 0.0 , delta: 0.0
sensorID = 1
ID NOT in sensors
new: -0.175949111938 , old: 0.0 , delta: 0.0
RUNNING DEMO
sensorID = 1
ID in sensors
new: -0.167

sensorID = 1
ID in sensors
new: -0.172551147461 , old: -0.163853302002 , delta: -0.00869784545898
sensorID = 1
ID in sensors
new: -0.180931503296 , old: -0.172551147461 , delta: -0.00838035583496
sensorID = 1
ID in sensors
new: -0.180931503296 , old: -0.180931503296 , delta: 0.0
sensorID = 1
ID in sensors
new: 0.0 , old: -0.180931503296 , delta: 0.180931503296
RUNNING DEMO
--------------EGM DATA--------------
(x:922.869628906, y:1.37901973724, z:1407.04064941)
RUNNING DEMO
--------------EGM DATA--------------
(x:922.870361328, y:1.37776637077, z:1407.04101562)
RUNNING DEMO
--------------EGM DATA--------------
(x:922.86907959, y:1.37980687618, z:1407.03979492)
RUNNING DEMO
--------------EGM DATA--------------
(x:922.869384766, y:1.37945258617, z:1407.0402832)
RUNNING DEMO
--------------EGM DATA--------------
(x:922.869750977, y:1.37887609005, z:1407.04077148)
RUNNING DEMO
--------------EGM DATA--------------
(x:922.870178223, y:1.37833726406, z:1407.04089355)
RUNNING DEMO
--------------